In [84]:
!pip install gymnasium

In [85]:
import gymnasium as gym
import numpy as np
import time
from IPython import display

In [86]:
env = gym.make('FrozenLake-v1', render_mode="ansi")

In [87]:
env.P[0][3] # Transition model

[(0.3333333333333333, 1, 0.0, False),
 (0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 0, 0.0, False)]

In [88]:
env.observation_space.n

16

In [89]:
env.action_space.n

4

In [90]:
def play(env, policy, render=False):
    state, _ = env.reset()
    total_reward = 0
    steps = 0
    done = False
    while not done:
        action = policy[state]
        next_state, reward, done, info, _ = env.step(action)
        total_reward += reward
        steps += 1
        if render:
            print(env.render())
            time.sleep(0.5)
            if not done:
                display.clear_output(wait=True)
        state = next_state

    return (total_reward, steps)

In [91]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play(env, policy_0)

(0.0, 9)

In [92]:

policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play(env, policy_0, True)


  (Left)
SFFF
FHFH
FFFH
HFFG



(0.0, 17)

In [93]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
play(env, policy_1, True)

  (Down)
SFFF
FHFH
FFFH
HFFG



(0.0, 3)

In [94]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
play(env, policy_2, True)

  (Down)
SFFF
FHFH
FFFH
HFFG



(0.0, 2)

In [95]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
play(env, policy_3, True)

  (Down)
SFFF
FHFH
FFFH
HFFG



(1.0, 19)

In [96]:
def play_multiple_times(env, policy, max_episodes):
    success = 0
    list_of_steps = []
    for i in range(max_episodes):
        total_reward, steps = play(env, policy)

        if total_reward > 0:
            success += 1
            list_of_steps.append(steps)

    print(f'Number of successes: {success}/{max_episodes}')
    print(f'Average number of steps: {np.mean(list_of_steps)}')

In [97]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play_multiple_times(env, policy_0, 1000)

Number of successes: 0/1000
Average number of steps: nan


In [98]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
play_multiple_times(env, policy_1, 1000)

Number of successes: 51/1000
Average number of steps: 11.882352941176471


In [99]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
play_multiple_times(env, policy_2, 1000)

Number of successes: 102/1000
Average number of steps: 15.480392156862745


In [100]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
play_multiple_times(env, policy_3, 1000)

Number of successes: 786/1000
Average number of steps: 43.03944020356234


In [101]:
def policy_evaluation(env, policy, max_iters=500, gamma=0.9):
    # Initialize the values of all states to be 0
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # Update the value of each state
        for state in range(env.observation_space.n):
            action = policy[state]

            # Compute the q-value of the action
            q_value = 0
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * prev_v_values[next_state])

            v_values[state] = q_value # update v-value

        # Check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            # print(f'Converged at {i}-th iteration.')
            break

    return v_values

In [102]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
v_values_0 = policy_evaluation(env, policy_0)
print(v_values_0)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [103]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
v_values_1 = policy_evaluation(env, policy_1)
print(v_values_1)

[0.01904157 0.01519815 0.03161906 0.02371389 0.02538879 0.
 0.06648515 0.         0.05924054 0.13822794 0.18999823 0.
 0.         0.21152109 0.56684236 0.        ]


In [104]:
np.all(v_values_1 >= v_values_0)

True

In [105]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
v_values_2 = policy_evaluation(env, policy_2)
print(v_values_2)

[0.02889625 0.01951972 0.03616977 0.0271268  0.04790519 0.
 0.07391985 0.         0.08288277 0.19339319 0.21022995 0.
 0.         0.35153135 0.62684674 0.        ]


In [106]:
np.all(v_values_2 >= v_values_1)

True

In [107]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
v_values_3 = policy_evaluation(env, policy_3)
print(v_values_3)

[0.06888666 0.06141097 0.07440714 0.05580443 0.09185068 0.
 0.11220679 0.         0.14543323 0.24749485 0.29961611 0.
 0.         0.37993438 0.63901935 0.        ]


In [108]:
np.all(v_values_3 >= v_values_2)

True

In [109]:
def value_iteration(env, max_iters=500, gamma=0.9):
    # initialize
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # update the v-value for each state
        for state in range(env.observation_space.n):
            q_values = []

            # compute the q-value for each action that we can perform at the state
            for action in range(env.action_space.n):
                q_value = 0
                # loop through each possible outcome
                for prob, next_state, reward, done in env.P[state][action]:
                    q_value += prob * (reward + gamma * prev_v_values[next_state])

                q_values.append(q_value)

            # select the max q-values
            best_action = np.argmax(q_values)
            v_values[state] = q_values[best_action]

        # check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break

    return v_values

In [110]:
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)

Converged at 79-th iteration.


In [111]:
optimal_v_values

array([0.06888615, 0.06141054, 0.07440682, 0.05580409, 0.09185022,
       0.        , 0.11220663, 0.        , 0.14543286, 0.2474946 ,
       0.29961593, 0.        , 0.        , 0.3799342 , 0.63901926,
       0.        ])

In [112]:
def policy_extraction(env, v_values, gamma=0.9):
    # initialize
    policy = np.zeros(env.observation_space.n, dtype=np.int32)

    # loop through each state in the environment
    for state in range(env.observation_space.n):
        q_values = []
        # loop through each action
        for action in range(env.action_space.n):
            q_value = 0
            # loop each possible outcome
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * v_values[next_state])

            q_values.append(q_value)

        # select the best action
        best_action = np.argmax(q_values)
        policy[state] = best_action

    return policy

In [113]:
optimal_policy = policy_extraction(env, optimal_v_values, gamma=0.9)

In [114]:
optimal_policy

array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0], dtype=int32)

In [115]:
play(env, optimal_policy, True)

  (Down)
SFFF
FHFH
FFFH
HFFG



(1.0, 92)

In [116]:
play_multiple_times(env, optimal_policy, 1000)

Number of successes: 779/1000
Average number of steps: 43.61232349165597


# Cài đặt Policy Iteration

In [117]:
def policyIteration(env, max_iters=1000, gamma=0.9):

    # Initialize the random policy
    policy = np.random.randint(env.action_space.n, size= (env.observation_space.n))
    # Initialize the value function to zeros
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
      # Copy the current policy to check for convergence later
      prev_policy= np.copy(policy)

      # policyEvaluation
      for j in range(max_iters):
          prev_v_values = np.copy(v_values)
          # Update the value of each state
          for state in range(env.observation_space.n):
              action = policy[state]
              # Compute the q-value of the action
              q_value = 0
              for prob, next_state, reward, done in env.P[state][action]:
                  q_value += prob * (reward + gamma * prev_v_values[next_state])
              v_values[state] = q_value # update v-value
          # Check convergence
          if np.all(np.isclose(v_values, prev_v_values)):
              break

      # policyExtraction
      for state in range(env.observation_space.n):
          q_values = []
          # loop through each action
          for action in range(env.action_space.n):
              q_value = 0
              # loop each possible outcome
              for prob, next_state, reward, done in env.P[state][action]:
                  q_value += prob * (reward + gamma * v_values[next_state])
              q_values.append(q_value)
          # select the best action
          best_action = np.argmax(q_values)
          policy[state] = best_action

      # Check if the policy has converged
      if np.all(prev_policy==policy):
          print(f'Converged at {i}-th iteration.')
          break

    return policy

# Tạo môi trường 'FrozenLake8x8-v1' và 'Taxi-v3'

In [118]:
envFrozenLake8x8 = gym.make('FrozenLake8x8-v1', render_mode="ansi")

envTaxiv3 = gym.make('Taxi-v3', render_mode="ansi")

In [119]:
import time
envs = np.array([env, envFrozenLake8x8, envTaxiv3])
envName = ['FrozenLake-v1', 'FrozenLake8x8-v1', 'Taxi-v3']
num = 7

# Tính thời gian trung bình của Value Iteration với mỗi môi trường

## Dùng cả Policy Extraction để đảm bảo Value Iteration và Policy Iteration được đánh giá trên cùng một tiêu chí

In [120]:
avg_vTime = []

for i, enviroment in enumerate(envs):
    print(envName[i], '\n')
    s = time.time()
    for i in range(num):
        optimalV = value_iteration(enviroment)
        policy_extraction(enviroment, optimalV)
    e = time.time()
    print('\n\n')
    avg_vTime.append((e-s)/num)

avg_vTime = np.array(avg_vTime)

FrozenLake-v1 

Converged at 79-th iteration.
Converged at 79-th iteration.
Converged at 79-th iteration.
Converged at 79-th iteration.
Converged at 79-th iteration.
Converged at 79-th iteration.
Converged at 79-th iteration.



FrozenLake8x8-v1 

Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.



Taxi-v3 

Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.





In [121]:
avg_pTime = []

for i, enviroment in enumerate(envs):
    print(envName[i], '\n')
    s = time.time()
    for i in range(num):
        policyIteration(enviroment)
    e = time.time()
    print('\n\n')
    avg_pTime.append((e-s)/num)

avg_pTime = np.array(avg_pTime)

FrozenLake-v1 

Converged at 2-th iteration.
Converged at 5-th iteration.
Converged at 3-th iteration.
Converged at 2-th iteration.
Converged at 5-th iteration.
Converged at 5-th iteration.
Converged at 3-th iteration.



FrozenLake8x8-v1 

Converged at 7-th iteration.
Converged at 9-th iteration.
Converged at 8-th iteration.
Converged at 9-th iteration.
Converged at 8-th iteration.
Converged at 8-th iteration.
Converged at 8-th iteration.



Taxi-v3 

Converged at 16-th iteration.
Converged at 15-th iteration.
Converged at 16-th iteration.
Converged at 16-th iteration.
Converged at 17-th iteration.
Converged at 17-th iteration.
Converged at 17-th iteration.





# So sánh **Value Iteration** với **Policy Iteration**

In [122]:
iter = ["Value Iteration", "Policy Iteration"]
for i in range(len(iter)):
    print(iter[i])
    for enviroment in envs:
        if not i:
            optimalV = value_iteration(enviroment)
            optimalPolicyV = policy_extraction(enviroment, optimalV)
        else:
            policyIteration(enviroment)
    print('\n\n\n')

Value Iteration
Converged at 79-th iteration.
Converged at 117-th iteration.
Converged at 116-th iteration.




Policy Iteration
Converged at 3-th iteration.
Converged at 7-th iteration.
Converged at 17-th iteration.






**Nhận xét**: Policy Iteration hội tụ nhanh hơn

In [123]:
envs = np.array([env, envFrozenLake8x8, envTaxiv3])
envName = ['FrozenLake-v1', 'FrozenLake8x8-v1', 'Taxi-v3']
iterations = ["Value Iteration", "Policy Iteration"]


for idx, environment in enumerate(envs):
    for iteration in iterations:
        print(envName[idx] + " dùng " + iteration)

        if iteration == "Value Iteration":
            optimalV = value_iteration(environment)
            optimalPolicyV = policy_extraction(environment, optimalV)

            play_multiple_times(environment, optimalPolicyV, 1000)
        else:
            optimalPolicyP = policyIteration(environment)

            play_multiple_times(environment, optimalPolicyP, 1000)
        print('\n\n')

FrozenLake-v1 dùng Value Iteration
Converged at 79-th iteration.
Number of successes: 785/1000
Average number of steps: 43.896815286624204



FrozenLake-v1 dùng Policy Iteration
Converged at 1-th iteration.
Number of successes: 776/1000
Average number of steps: 43.71005154639175



FrozenLake8x8-v1 dùng Value Iteration
Converged at 117-th iteration.
Number of successes: 753/1000
Average number of steps: 73.77822045152722



FrozenLake8x8-v1 dùng Policy Iteration
Converged at 2-th iteration.
Number of successes: 756/1000
Average number of steps: 73.16931216931216



Taxi-v3 dùng Value Iteration
Converged at 116-th iteration.
Number of successes: 1000/1000
Average number of steps: 13.054



Taxi-v3 dùng Policy Iteration
Converged at 16-th iteration.
Number of successes: 1000/1000
Average number of steps: 13.07





**Nhận xét**: Ta thấy cả 2 Iteration đều cho kết quả tối ưu và có số bước đi trung bình gần như bằng nhau đối với mỗi môi trường

In [124]:
iterations = ["Value Iteration", "Policy Iteration"]
for iter in range(len(iterations)):
    time = avg_vTime.mean() if iter == 0 else avg_pTime.mean()
    print(f"Thời gian chạy của {iterations[iter]}:", time, '\n')


Thời gian chạy của Value Iteration: 2.4531164509909495 

Thời gian chạy của Policy Iteration: 1.4052342346736362 



**Nhận xét:** Policy Iteration có thời gian chạy trung bình nhanh hơn Value Iteration